In [4]:
import sys
import os

# 添加 factor_evaluation.pyd 所在的目录到系统路径
pyd_path = r"D:\MG\！internship\！4L CAPITAL\因子评估器"
sys.path.append(pyd_path)

from factor_evaluation import FactorEvaluation
from factor_evaluation import DataService
import numpy as np
import pandas as pd
from numpy import abs, log, sign
from scipy.stats import rankdata

In [5]:
# 辅助函数定义
def returns(df):
    return df.rolling(2).apply(lambda x: x.iloc[-1] / x.iloc[0]) - 1

def ts_sum(df, window=10):
    return df.rolling(window).sum()

def sma(df, window=10):
    return df.rolling(window).mean()

def stddev(df, window=10):
    return df.rolling(window).std()

def correlation(x, y, window=10):
    return x.rolling(window).corr(y).fillna(0).replace([np.inf, -np.inf], 0)

def covariance(x, y, window=10):
    return x.rolling(window).cov(y)

def rolling_rank(na):
    return rankdata(na, method='min')[-1]

def ts_rank(df, window=10):
    return df.rolling(window).apply(rolling_rank)

def rolling_prod(na):
    return np.prod(na)

def product(df, window=10):
    return df.rolling(window).apply(rolling_prod)

def ts_min(df, window=10):
    return df.rolling(window).min()

def ts_max(df, window=10):
    return df.rolling(window).max()

def delta(df, period=1):
    return df.diff(period)

def delay(df, period=1):
    return df.shift(period)

def rank(df):
    # 在时间序列上的滚动排名
    return df.rolling(20).apply(lambda x: (x.rank(pct=True).iloc[-1]))

def scale(df, k=1):
    return df.mul(k).div(np.abs(df).sum())

def ts_argmax(df, window=10):
    return df.rolling(window).apply(np.argmax) + 1 

def ts_argmin(df, window=10):
    return df.rolling(window).apply(np.argmin) + 1

def decay_linear(df, period=10):
    weights = np.array(range(1, period+1))
    sum_weights = np.sum(weights)
    return df.rolling(period).apply(lambda x: np.sum(weights*x) / sum_weights)

def max(sr1, sr2):
    return np.maximum(sr1, sr2)

def min(sr1, sr2):
    return np.minimum(sr1, sr2)

In [6]:
# 因子实现类
class AlphasETH:
    def __init__(self, df_data):
        self.open = df_data['open']
        self.high = df_data['high']
        self.low = df_data['low']
        self.close = df_data['close']
        self.volume = df_data['volume']
        self.returns = returns(self.close)
        self.vwap = df_data['vwap']

    def alpha001(self):
        inner = self.close.copy()
        inner[self.returns < 0] = stddev(self.returns, 20)
        return rank(ts_argmax(inner ** 2, 5)) - 0.5
    
    def alpha002(self):
        df = -1 * correlation(rank(delta(log(self.volume), 2)), rank((self.close - self.open) / self.open), 6)
        return df.replace([-np.inf, np.inf], 0).fillna(value=0)
    
    def alpha003(self):
        df = -1 * correlation(rank(self.open), rank(self.volume), 10)
        return df.replace([-np.inf, np.inf], 0).fillna(value=0)
    
    def alpha004(self):
        return -1 * ts_rank(rank(self.low), 9)
    
    def alpha005(self):
        return (rank((self.open - (ts_sum(self.vwap, 10) / 10)))) * (-1 * abs(rank((self.close - self.vwap))))
    
    def alpha006(self):
        df = -1 * correlation(self.open, self.volume, 10)
        return df.replace([-np.inf, np.inf], 0).fillna(value=0)
    
    def alpha007(self):
        adv20 = sma(self.volume, 20)
        alpha = -1 * ts_rank(abs(delta(self.close, 7)), 60) * sign(delta(self.close, 7))
        alpha[adv20 >= self.volume] = -1
        return alpha
    
    def alpha008(self):
        return -1 * (rank(((ts_sum(self.open, 5) * ts_sum(self.returns, 5)) - 
                          delay((ts_sum(self.open, 5) * ts_sum(self.returns, 5)), 10))))
    
    def alpha009(self):
        delta_close = delta(self.close, 1)
        cond_1 = ts_min(delta_close, 5) > 0
        cond_2 = ts_max(delta_close, 5) < 0
        alpha = -1 * delta_close
        alpha[cond_1 | cond_2] = delta_close
        return alpha
    
    def alpha010(self):
        delta_close = delta(self.close, 1)
        cond_1 = ts_min(delta_close, 4) > 0
        cond_2 = ts_max(delta_close, 4) < 0
        alpha = -1 * delta_close
        alpha[cond_1 | cond_2] = delta_close
        return rank(alpha)
    
    def alpha011(self):
        return ((rank(ts_max((self.vwap - self.close), 3)) + 
                rank(ts_min((self.vwap - self.close), 3))) * 
                rank(delta(self.volume, 3)))
    
    def alpha012(self):
        return sign(delta(self.volume, 1)) * (-1 * delta(self.close, 1))

    def alpha013(self):
        return -1 * rank(covariance(rank(self.close), rank(self.volume), 5))
    
    def alpha014(self):
        df = correlation(self.open, self.volume, 10)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * rank(delta(self.returns, 3)) * df
    
    def alpha015(self):
        df = correlation(rank(self.high), rank(self.volume), 3)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * ts_sum(rank(df), 3)
    
    def alpha016(self):
        return -1 * rank(covariance(rank(self.high), rank(self.volume), 5))
    
    def alpha017(self):
        adv20 = sma(self.volume, 20)
        return -1 * (rank(ts_rank(self.close, 10)) *
                     rank(delta(delta(self.close, 1), 1)) *
                     rank(ts_rank((self.volume / adv20), 5)))
        
    def alpha018(self):
        df = correlation(self.close, self.open, 10)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * (rank((stddev(abs((self.close - self.open)), 5) + 
                          (self.close - self.open)) + df))
    
    def alpha019(self):
        return ((-1 * sign((self.close - delay(self.close, 7)) + delta(self.close, 7))) *
                (1 + rank(1 + ts_sum(self.returns, 250))))
    
    def alpha020(self):
        return -1 * (rank(self.open - delay(self.high, 1)) *
                     rank(self.open - delay(self.close, 1)) *
                     rank(self.open - delay(self.low, 1)))

    def alpha021(self):
        cond_1 = sma(self.close, 8) + stddev(self.close, 8) < sma(self.close, 2)
        cond_2 = sma(self.close, 2) < sma(self.close, 8) - stddev(self.close, 8)
        cond_3 = sma(self.volume, 20) / self.volume < 1
        return (cond_1 | ((~cond_1) & (~cond_2) & (~cond_3))).astype('int')*(-2)+1
    
    def alpha022(self):
        df = correlation(self.high, self.volume, 5)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * delta(df, 5) * rank(stddev(self.close, 20))

    def alpha023(self):
        cond = sma(self.high, 20) < self.high
        alpha = self.close.copy(deep=True)
        alpha[cond] = -1 * delta(self.high, 2).fillna(value=0)
        alpha[~cond] = 0
        return alpha
    
    def alpha024(self):
        cond = delta(sma(self.close, 100), 100) / delay(self.close, 100) <= 0.05
        alpha = -1 * delta(self.close, 3)
        alpha[cond] = -1 * (self.close - ts_min(self.close, 100))
        return alpha
    
    def alpha025(self):
        adv20 = sma(self.volume, 20)
        return rank(((((-1 * self.returns) * adv20) * self.vwap) * (self.high - self.close)))
    
    def alpha026(self):
        df = correlation(ts_rank(self.volume, 5), ts_rank(self.high, 5), 5)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * ts_max(df, 3)
    
    def alpha027(self):
        alpha = rank((sma(correlation(rank(self.volume), rank(self.vwap), 6), 2) / 2.0))
        return sign((alpha - 0.5) * (-2))
    
    def alpha028(self):
        adv20 = sma(self.volume, 20)
        df = correlation(adv20, self.low, 5)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return scale(((df + ((self.high + self.low) / 2)) - self.close))

    def alpha029(self):
        return (ts_min(rank(rank(scale(log(ts_sum(rank(rank(-1 * rank(delta((self.close - 1), 5)))), 2))))), 5) +
                ts_rank(delay((-1 * self.returns), 6), 5))

    def alpha030(self):
        delta_close = delta(self.close, 1)
        inner = sign(delta_close) + sign(delay(delta_close, 1)) + sign(delay(delta_close, 2))
        return ((1.0 - rank(inner)) * ts_sum(self.volume, 5)) / ts_sum(self.volume, 20)

    def alpha031(self):
        adv20 = sma(self.volume, 20)
        df = correlation(adv20, self.low, 12).replace([-np.inf, np.inf], 0).fillna(value=0)         
        p1 = rank(rank(rank(decay_linear((-1 * rank(rank(delta(self.close, 10))), 10)))))
        p2 = rank((-1 * delta(self.close, 3)))
        p3 = sign(scale(df))
        return p1 + p2 + p3

    def alpha032(self):
        return scale(((sma(self.close, 7) / 7) - self.close)) + (20 * scale(correlation(self.vwap, delay(self.close, 5), 230)))
    
    def alpha033(self):
        return rank(-1 + (self.open / self.close))
    
    def alpha034(self):
        inner = stddev(self.returns, 2) / stddev(self.returns, 5)
        inner = inner.replace([-np.inf, np.inf], 1).fillna(value=1)
        return rank(2 - rank(inner) - rank(delta(self.close, 1)))

    def alpha035(self):
        return ((ts_rank(self.volume, 32) *
                (1 - ts_rank(self.close + self.high - self.low, 16))) *
                (1 - ts_rank(self.returns, 32)))
            
    def alpha036(self):
        adv20 = sma(self.volume, 20)
        return (((((2.21 * rank(correlation((self.close - self.open), delay(self.volume, 1), 15))) + 
                (0.7 * rank((self.open - self.close)))) + 
                (0.73 * rank(ts_rank(delay((-1 * self.returns), 6), 5)))) + 
                rank(abs(correlation(self.vwap, adv20, 6)))) + 
                (0.6 * rank((((sma(self.close, 200) / 200) - self.open) * (self.close - self.open)))))
    
    def alpha037(self):
        return rank(correlation(delay(self.open - self.close, 1), self.close, 200)) + rank(self.open - self.close)
    
    def alpha038(self):
        inner = self.close / self.open
        inner = inner.replace([-np.inf, np.inf], 1).fillna(value=1)
        return -1 * rank(ts_rank(self.open, 10)) * rank(inner)
    
    def alpha039(self):
        adv20 = sma(self.volume, 20)
        return ((-1 * rank(delta(self.close, 7) * (1 - rank(decay_linear((self.volume / adv20), 9))))) *
                (1 + rank(sma(self.returns, 250))))
    
    def alpha040(self):
        return -1 * rank(stddev(self.high, 10)) * correlation(self.high, self.volume, 10)

    def alpha041(self):
        return pow((self.high * self.low), 0.5) - self.vwap
    
    def alpha042(self):
        return rank((self.vwap - self.close)) / rank((self.vwap + self.close))
        
    def alpha043(self):
        adv20 = sma(self.volume, 20)
        return ts_rank(self.volume / adv20, 20) * ts_rank((-1 * delta(self.close, 7)), 8)

    def alpha044(self):
        df = correlation(self.high, rank(self.volume), 5)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * df

    def alpha045(self):
        df = correlation(self.close, self.volume, 2)
        df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
        return -1 * (rank(sma(delay(self.close, 5), 20)) * df *
                     rank(correlation(ts_sum(self.close, 5), ts_sum(self.close, 20), 2)))
    
    def alpha046(self):
        inner = ((delay(self.close, 20) - delay(self.close, 10)) / 10) - ((delay(self.close, 10) - self.close) / 10)
        alpha = (-1 * delta(self.close))
        alpha[inner < 0] = 1
        alpha[inner > 0.25] = -1
        return alpha

    def alpha047(self):
        adv20 = sma(self.volume, 20)
        return ((((rank((1 / self.close)) * self.volume) / adv20) * 
                ((self.high * rank((self.high - self.close))) / (sma(self.high, 5) / 5))) - 
                rank((self.vwap - delay(self.vwap, 5))))
    
    # Alpha#48 需要行业中性化，跳过
    def alpha048(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha049(self):
        inner = (((delay(self.close, 20) - delay(self.close, 10)) / 10) - ((delay(self.close, 10) - self.close) / 10))
        alpha = (-1 * delta(self.close))
        alpha[inner < -0.1] = 1
        return alpha
    
    def alpha050(self):
        return (-1 * ts_max(rank(correlation(rank(self.volume), rank(self.vwap), 5)), 5))
    
    def alpha051(self):
        inner = (((delay(self.close, 20) - delay(self.close, 10)) / 10) - ((delay(self.close, 10) - self.close) / 10))
        alpha = (-1 * delta(self.close))
        alpha[inner < -0.05] = 1
        return alpha
    
    def alpha052(self):
        return (((-1 * delta(ts_min(self.low, 5), 5)) *
                rank(((ts_sum(self.returns, 240) - ts_sum(self.returns, 20)) / 220))) * ts_rank(self.volume, 5))
        
    def alpha053(self):
        inner = (self.close - self.low).replace(0, 0.0001)
        return -1 * delta((((self.close - self.low) - (self.high - self.close)) / inner), 9)

    def alpha054(self):
        inner = (self.low - self.high).replace(0, -0.0001)
        return -1 * (self.low - self.close) * (self.open ** 5) / (inner * (self.close ** 5))

    def alpha055(self):
        divisor = (ts_max(self.high, 12) - ts_min(self.low, 12)).replace(0, 0.0001)
        inner = (self.close - ts_min(self.low, 12)) / (divisor)
        df = correlation(rank(inner), rank(self.volume), 6)
        return -1 * df.replace([-np.inf, np.inf], 0).fillna(value=0)

    # Alpha#56 需要市值数据，跳过
    def alpha056(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha057(self):
        return (0 - (1 * ((self.close - self.vwap) / decay_linear(rank(ts_argmax(self.close, 30)), 2))))
    
    # Alpha#58 需要行业中性化，跳过
    def alpha058(self):
        return pd.Series(0, index=self.close.index)
    
    # Alpha#59 需要行业中性化，跳过
    def alpha059(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha060(self):
        divisor = (self.high - self.low).replace(0, 0.0001)
        inner = ((self.close - self.low) - (self.high - self.close)) * self.volume / divisor
        return - ((2 * scale(rank(inner))) - scale(rank(ts_argmax(self.close, 10))))
    
    def alpha061(self):
        adv180 = sma(self.volume, 180)
        return (rank((self.vwap - ts_min(self.vwap, 16))) < rank(correlation(self.vwap, adv180, 18))).astype('int')
    
    def alpha062(self):
        adv20 = sma(self.volume, 20)
        return ((rank(correlation(self.vwap, sma(adv20, 22), 10)) < 
                rank(((rank(self.open) + rank(self.open)) < 
                     (rank(((self.high + self.low) / 2)) + rank(self.high))))) * -1)
    
    # Alpha#63 需要行业中性化，跳过
    def alpha063(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha064(self):
        adv120 = sma(self.volume, 120)
        return ((rank(correlation(sma(((self.open * 0.178404) + (self.low * (1 - 0.178404))), 13),
                     sma(adv120, 13), 17)) < 
                rank(delta(((((self.high + self.low) / 2) * 0.178404) + 
                           (self.vwap * (1 - 0.178404))), 4))) * -1)    
    def alpha065(self):
        adv60 = sma(self.volume, 60)
        return ((rank(correlation(((self.open * 0.00817205) + (self.vwap * (1 - 0.00817205))), 
                     sma(adv60, 9), 6))) < 
                rank((self.open - ts_min(self.open, 14)))) * -1
      
    def alpha066(self):
        return ((rank(decay_linear(delta(self.vwap, 4), 7)) + 
                ts_rank(decay_linear(((((self.low * 0.96633) + (self.low * (1 - 0.96633))) - 
                                     self.vwap) / (self.open - ((self.high + self.low) / 2)), 11), 7)) * -1))
    
    # Alpha#67 需要行业中性化，跳过
    def alpha067(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha068(self):
        adv15 = sma(self.volume, 15)
        return ((ts_rank(correlation(rank(self.high), rank(adv15), 9), 14) <
                rank(delta(((self.close * 0.518371) + (self.low * (1 - 0.518371))), 2)) * 14) * -1)
     
    # Alpha#69 需要行业中性化，跳过
    def alpha069(self):
        return pd.Series(0, index=self.close.index)
         
    # Alpha#70 需要行业中性化，跳过
    def alpha070(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha071(self):
        adv180 = sma(self.volume, 180)
        p1 = ts_rank(decay_linear(correlation(ts_rank(self.close, 3), ts_rank(adv180, 12), 18), 4), 16)
        p2 = ts_rank(decay_linear((rank(((self.low + self.open) - (self.vwap + self.vwap))).pow(2)), 16), 4)
        return max(p1, p2)
    
    def alpha072(self):
        adv40 = sma(self.volume, 40)
        return (rank(decay_linear(correlation(((self.high + self.low) / 2), adv40, 9), 10)) /\
               rank(decay_linear(correlation(ts_rank(self.vwap, 4), ts_rank(self.volume, 19), 7), 3)))
    
    def alpha073(self):
        p1 = rank(decay_linear(delta(self.vwap, 5), 3))
        p2 = ts_rank(decay_linear(((delta(((self.open * 0.147155) + 
                                         (self.low * (1 - 0.147155))), 2) / 
                                  ((self.open * 0.147155) + 
                                   (self.low * (1 - 0.147155)))) * -1), 3), 17)
        return -1 * max(p1, p2)
    
    def alpha074(self):
        adv30 = sma(self.volume, 30)
        return ((rank(correlation(self.close, sma(adv30, 37), 15)) <
                rank(correlation(rank(((self.high * 0.0261661) + 
                                      (self.vwap * (1 - 0.0261661))), 
                                rank(self.volume), 11))) * -1))
    
    def alpha075(self):
        adv50 = sma(self.volume, 50)
        return (rank(correlation(self.vwap, self.volume, 4)) < 
                rank(correlation(rank(self.low), rank(adv50), 12))).astype('int')
    
    # Alpha#76 需要行业中性化，跳过
    def alpha076(self):
        return pd.Series(0, index=self.close.index)
     
    def alpha077(self):
        adv40 = sma(self.volume, 40)
        p1 = rank(decay_linear(((((self.high + self.low) / 2) + self.high) - 
                               (self.vwap + self.high)), 20))
        p2 = rank(decay_linear(correlation(((self.high + self.low) / 2), adv40, 3), 6))
        return min(p1, p2)
    
    def alpha078(self):
        adv40 = sma(self.volume, 40)
        return (rank(correlation(ts_sum(((self.low * 0.352233) + 
                                        (self.vwap * (1 - 0.352233))), 20),
                     ts_sum(adv40, 20), 7)).pow(rank(correlation(rank(self.vwap), 
                                                                rank(self.volume), 6))))
    
    # Alpha#79 需要行业中性化，跳过
    def alpha079(self):
        return pd.Series(0, index=self.close.index)
     
    # Alpha#80 需要行业中性化，跳过
    def alpha080(self):
        return pd.Series(0, index=self.close.index)
     
    def alpha081(self):
        adv10 = sma(self.volume, 10)
        return ((rank(log(product(rank((rank(correlation(self.vwap, ts_sum(adv10, 50), 8)).pow(4))), 15)) < 
                rank(correlation(rank(self.vwap), rank(self.volume), 5))) * -1))
    
    # Alpha#82 需要行业中性化，跳过
    def alpha082(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha083(self):
        return ((rank(delay(((self.high - self.low) / (ts_sum(self.close, 5) / 5)), 2)) * 
                rank(rank(self.volume))) / (((self.high - self.low) / 
                                           (ts_sum(self.close, 5) / 5)) / 
                                          (self.vwap - self.close)))
    
    def alpha084(self):
        return pow(ts_rank((self.vwap - ts_max(self.vwap, 15)), 21), delta(self.close, 5))
    
    def alpha085(self):
        adv30 = sma(self.volume, 30)
        return (rank(correlation(((self.high * 0.876703) + 
                                (self.close * (1 - 0.876703))), adv30, 10)).
                pow(rank(correlation(ts_rank(((self.high + self.low) / 2), 4), 
                                   ts_rank(self.volume, 10), 7))))
    
    def alpha086(self):
        adv20 = sma(self.volume, 20)
        return ((ts_rank(correlation(self.close, sma(adv20, 15), 6), 20) < 
                rank(((self.open + self.close) - (self.vwap + self.open))) * 20) * -1)
    
    # Alpha#87 需要行业中性化，跳过
    def alpha087(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha088(self):
        adv60 = sma(self.volume, 60)
        p1 = rank(decay_linear(((rank(self.open) + rank(self.low)) - 
                              (rank(self.high) + rank(self.close))), 8))
        p2 = ts_rank(decay_linear(correlation(ts_rank(self.close, 8), 
                                 ts_rank(adv60, 21), 8), 7), 3)
        return min(p1, p2)
    
    # Alpha#89 需要行业中性化，跳过
    def alpha089(self):
        return pd.Series(0, index=self.close.index)
     
    # Alpha#90 需要行业中性化，跳过
    def alpha090(self):
        return pd.Series(0, index=self.close.index)
     
    # Alpha#91 需要行业中性化，跳过
    def alpha091(self):
        return pd.Series(0, index=self.close.index)
     
    def alpha092(self):
        adv30 = sma(self.volume, 30)
        p1 = ts_rank(decay_linear(((((self.high + self.low) / 2) + 
                                  self.close) < (self.low + self.open)), 15), 19)
        p2 = ts_rank(decay_linear(correlation(rank(self.low), rank(adv30), 8), 7), 7)
        return min(p1, p2)
    
    # Alpha#93 需要行业中性化，跳过
    def alpha093(self):
        return pd.Series(0, index=self.close.index)
     
    def alpha094(self):
        adv60 = sma(self.volume, 60)
        return ((rank((self.vwap - ts_min(self.vwap, 12))).
                pow(ts_rank(correlation(ts_rank(self.vwap, 20), 
                                      ts_rank(adv60, 4), 18), 3)) * -1))
    
    def alpha095(self):
        adv40 = sma(self.volume, 40)
        return (rank((self.open - ts_min(self.open, 12))) * 12 < 
                ts_rank((rank(correlation(sma(((self.high + self.low) / 2), 19), 
                                        sma(adv40, 19), 13)).pow(5)), 12)).astype('int')
    
    def alpha096(self):
        adv60 = sma(self.volume, 60)
        p1 = ts_rank(decay_linear(correlation(rank(self.vwap), rank(self.volume), 4), 4), 8)
        p2 = ts_rank(decay_linear(ts_argmax(correlation(ts_rank(self.close, 7), 
                                                     ts_rank(adv60, 4), 4), 13), 14), 13)
        return -1 * max(p1, p2)
    
    # Alpha#97 需要行业中性化，跳过
    def alpha097(self):
        return pd.Series(0, index=self.close.index)
    
    def alpha098(self):
        adv5 = sma(self.volume, 5)
        adv15 = sma(self.volume, 15)
        return (rank(decay_linear(correlation(self.vwap, sma(adv5, 26), 5), 7)) -
                rank(decay_linear(ts_rank(ts_argmin(correlation(rank(self.open), 
                                                             rank(adv15), 21), 9), 7), 8)))
    
    def alpha099(self):
        adv60 = sma(self.volume, 60)
        return ((rank(correlation(ts_sum(((self.high + self.low) / 2), 20), 
                                ts_sum(adv60, 20), 9)) <
                rank(correlation(self.low, self.volume, 6))) * -1)
    
    # Alpha#100 需要行业中性化，跳过
    def alpha100(self):
        return pd.Series(0, index=self.close.index)

    def alpha101(self):
        return (self.close - self.open) / ((self.high - self.low) + 0.001)

In [7]:
# 主程序
if __name__ == "__main__":
    # 获取数据
    ds = DataService()
    df = ds['ETHUSDT_15m_2020_2025']['2021-10-01':]
    
    # 创建因子评估器
    evaluator = FactorEvaluation(df=df, future_return_periods=10)
    
    # 创建因子计算器
    alpha_calculator = AlphasETH(df)
    
    # 评估所有因子
    for i in range(1, 102):
        alpha_name = f"alpha{i:03d}"
        if hasattr(alpha_calculator, alpha_name):
            print(f"Evaluating {alpha_name}...")
            
            # 获取因子计算函数
            alpha_func = getattr(alpha_calculator, alpha_name)
            
            try:
                # 计算因子值
                factor_values = alpha_func()
                
                # 设置因子进行评估
                evaluator.set_factor(
                    factor_data_or_func=factor_values,
                    factor_name=alpha_name
                )
                
                # 运行评估
                result = evaluator.run_full_evaluation(run_stationarity_test=True)
                print(f"Evaluation completed for {alpha_name}")
            except Exception as e:
                print(f"Error evaluating {alpha_name}: {str(e)}")
        else:
            print(f"Skipping {alpha_name} (not implemented)")

KeyError: 'vwap'